### Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import shap
import sklearn
import timeit
import seaborn as sns
import os
from pathlib import Path
import warnings

dataPath = str(Path(os.getcwd()).parent.absolute())+"/data"
figuresPath = str(Path(os.getcwd()).parent.absolute())+"/reports/figures"

In [5]:
def significance(x):
    if x>0.1:return("")
    elif x>0.05:return("*")
    elif x>0.01:return("**")
    else:return("***")

In [6]:
sectors = pd.read_excel(dataPath+'/raw/sectors.xlsx', index_col=0, sheet_name="Sheet2") 

In [7]:
GMB_MKT_daily = pd.read_csv(dataPath+'/interim/marketGreenPortfolios.csv', index_col=0).loc[:,["MKT_GMB_BL|DAILY_RETURNS","MKT_GMB_RO|DAILY_RETURNS","MKT_GMB|DAILY_RETURNS"]]
# GMB_MKT_daily.head()
GMB_INDW_daily = pd.read_csv(dataPath+'/interim/industryWeightedGreenPortfolios.csv', index_col=0)[['IND_W_GREEN|DAILY_RETURNS','IND_W_BROWN|DAILY_RETURNS',"IND_W_GMB_BL|DAILY_RETURNS","IND_W_GMB_RO|DAILY_RETURNS","IND_W_GMB|DAILY_RETURNS"]]
#  GMB_INDW_daily.head()
MKT_daily = pd.read_csv(dataPath+'/interim/marketData.csv', index_col=0)["MKT|DAILY_RETURNS"]
# MKT_daily.head()
STOCKS_daily = pd.read_csv(dataPath+'/raw/stocksData.csv', index_col=0) 
#  STOCKS_daily.head()
FF_daily = pd.read_csv(dataPath+'/raw/ffData.csv', index_col=0)
# ff_ret.head()

G_INDEXES_daily = pd.read_csv(dataPath+'/raw/greenIndexes.csv', index_col=0) 
# G_INDEXES_daily.head()

IND_daily = pd.DataFrame()
for sector in sectors.index:
    IND_daily[str(sector)+"|DAILY_RETURNS"]=pd.read_csv(dataPath+"/interim/industries/industry"+str(sector)+'MarketData.csv', index_col=0)["MKT|DAILY_RETURNS"]
    IND_daily[str(sector)+"_BROWN_BL|DAILY_RETURNS"]=pd.read_csv(dataPath+"/interim/industries/industry"+str(sector)+'GreenPortfolios.csv', index_col=0)["MKT_BROWN_BL|DAILY_RETURNS"]
    IND_daily[str(sector)+"_BROWN_RO|DAILY_RETURNS"]=pd.read_csv(dataPath+"/interim/industries/industry"+str(sector)+'GreenPortfolios.csv', index_col=0)["MKT_BROWN_RO|DAILY_RETURNS"]
    IND_daily[str(sector)+"_BROWN|DAILY_RETURNS"] = (IND_daily[str(sector)+"_BROWN_RO|DAILY_RETURNS"]+IND_daily[str(sector)+"_BROWN_BL|DAILY_RETURNS"])/2
    IND_daily[str(sector)+"_GREEN_BL|DAILY_RETURNS"]=pd.read_csv(dataPath+"/interim/industries/industry"+str(sector)+'GreenPortfolios.csv', index_col=0)["MKT_GREEN_BL|DAILY_RETURNS"]
    IND_daily[str(sector)+"_GREEN_RO|DAILY_RETURNS"]=pd.read_csv(dataPath+"/interim/industries/industry"+str(sector)+'GreenPortfolios.csv', index_col=0)["MKT_GREEN_RO|DAILY_RETURNS"]
    IND_daily[str(sector)+"_GREEN|DAILY_RETURNS"] = (IND_daily[str(sector)+"_GREEN_RO|DAILY_RETURNS"]+IND_daily[str(sector)+"_GREEN_BL|DAILY_RETURNS"])/2



In [8]:
sectors = pd.read_excel(dataPath+'/raw/sectors.xlsx', index_col=0, sheet_name="Sheet2") 

In [9]:
G_INDEXES_daily

,SPX|DAILY_RETURNS,QGREEN|DAILY_RETURNS,F3OILG|DAILY_RETURNS,SXEP|DAILY_RETURNS,SXWESGU|DAILY_RETURNS,RENIXX|DAILY_RETURNS,CELS|DAILY_RETURNS,ECO|DAILY_RETURNS,SPGTCED|DAILY_RETURNS,DJ_OIL_AND_GAS|DAILY_RETURNS
2016-09-13,-1.48,-1.05,-1.53,-2.82,-0.99,-1.09,-1.96,-2.90,-1.59,-2.94
2016-09-14,-0.06,-0.13,-0.83,-0.68,-0.18,-0.67,-0.90,-1.61,-1.30,-1.21
2016-09-15,1.01,0.67,0.27,0.27,0.51,0.00,1.35,0.82,0.37,1.11
2016-09-16,-0.38,-0.54,-0.34,-1.39,-0.96,-0.52,-0.01,-0.60,-0.50,-0.83
2016-09-19,0.00,0.82,1.69,1.45,1.02,0.99,1.10,1.13,1.46,-0.08
...,...,...,...,...,...,...,...,...,...,...
2022-11-29,-0.16,-0.59,1.63,1.76,-0.29,-1.02,-0.61,-0.13,-0.62,1.22
2022-11-30,3.09,2.94,1.57,1.66,0.16,0.66,5.37,6.02,2.90,0.72
2022-12-01,-0.09,0.99,-2.16,-0.98,2.36,2.15,-1.78,-2.61,0.35,-0.52
2022-12-02,-0.12,0.02,-1.11,-1.06,-0.19,0.95,1.66,2.65,0.80,-0.59


In [10]:
#creating dummy for covid
FF_daily['COV1_1Jan2021'] = FF_daily.loc[:, 'Mkt-RF']
FF_daily['COV2_1Jan2021'] = FF_daily.loc[:, 'Mkt-RF']

# FF_daily.loc[:'2020-11-03', 'COV1_1Jan2021'] = 0  #Biden election
# FF_daily.loc['2020-11-04':, 'COV1_1Jan2021'] = 1
# FF_daily.loc[:'2020-11-03', 'COV2_1Jan2021'] = 0

# FF_daily.loc[:'2021-01-01', 'COV1_1Jan2021'] = 0  #1st january 2021 (end of covid repercussions ?)
# FF_daily.loc['2021-01-02':, 'COV1_1Jan2021'] = 1
# FF_daily.loc[:'2021-01-04', 'COV2_1Jan2021'] = 0

# FF_daily.loc[:'2020-01-31', 'COV1_1Jan2021'] = 0  # COVID starts (WHO Issues Global Health Emergency)
# FF_daily.loc['2020-02-01':, 'COV1_1Jan2021'] = 1
# FF_daily.loc[:'2020-01-31', 'COV2_1Jan2021'] = 0

FF_daily.loc[:'2020-03-11', 'COV1_1Jan2021'] = 0  # COVID starts nb2 (WHO Declares COVID-19 a Pandemic)
FF_daily.loc['2020-03-12':, 'COV1_1Jan2021'] = 1
FF_daily.loc[:'2020-03-11', 'COV2_1Jan2021'] = 0

# FF_daily.loc[:, 'COV1_1Jan2021'] = 0  # No dummy variables
# FF_daily.loc[:, 'COV2_1Jan2021'] = 0

# FF_daily.head()
# FF_daily.tail()


In [11]:
merged=pd.concat([FF_daily,MKT_daily,IND_daily,GMB_MKT_daily,GMB_INDW_daily,G_INDEXES_daily],axis=1)
merged=merged.dropna()# drops the first row which contains NaN
# merged.head()

In [12]:
merged

,Mkt-RF,SMB,HML,RMW,CMA,RF,COV1_1Jan2021,COV2_1Jan2021,MKT|DAILY_RETURNS,10|DAILY_RETURNS,...,SPX|DAILY_RETURNS,QGREEN|DAILY_RETURNS,F3OILG|DAILY_RETURNS,SXEP|DAILY_RETURNS,SXWESGU|DAILY_RETURNS,RENIXX|DAILY_RETURNS,CELS|DAILY_RETURNS,ECO|DAILY_RETURNS,SPGTCED|DAILY_RETURNS,DJ_OIL_AND_GAS|DAILY_RETURNS
2016-09-14,-0.08,0.06,-0.81,0.09,-0.45,0.001,0.0,0.00,-0.073768,-1.066312,...,-0.06,-0.13,-0.83,-0.68,-0.18,-0.67,-0.90,-1.61,-1.30,-1.21
2016-09-15,1.08,0.30,-0.32,0.06,-0.21,0.001,0.0,0.00,1.057124,0.982844,...,1.01,0.67,0.27,0.27,0.51,0.00,1.35,0.82,0.37,1.11
2016-09-16,-0.36,0.29,-0.45,0.21,-0.19,0.001,0.0,0.00,-0.325997,-0.980666,...,-0.38,-0.54,-0.34,-1.39,-0.96,-0.52,-0.01,-0.60,-0.50,-0.83
2016-09-19,0.05,0.51,0.15,-0.08,0.36,0.001,0.0,0.00,-0.038256,-0.014394,...,0.00,0.82,1.69,1.45,1.02,0.99,1.10,1.13,1.46,-0.08
2016-09-20,-0.02,-0.35,-0.41,-0.55,-0.10,0.001,0.0,0.00,0.106607,-0.862621,...,0.03,-0.18,-0.51,-0.90,-0.17,0.65,-0.64,0.18,0.29,-0.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,1.78,1.02,-1.52,-1.16,-1.23,0.011,1.0,1.78,1.626782,-0.057130,...,1.63,2.83,-1.82,-0.52,2.22,2.58,5.11,6.60,2.55,0.11
2022-10-26,-0.66,0.69,0.89,-0.81,0.69,0.011,1.0,-0.66,-1.403603,1.311725,...,-0.74,1.16,0.41,0.94,1.55,4.09,1.59,0.08,2.83,1.30
2022-10-27,-0.54,0.43,1.06,-0.38,0.51,0.011,1.0,-0.54,-0.863706,0.259050,...,-0.61,-0.17,4.66,3.54,-0.01,1.53,-0.56,-0.71,0.89,0.29
2022-10-28,2.33,0.05,-0.34,0.38,0.60,0.011,1.0,2.33,2.355851,0.348592,...,2.46,0.99,-0.85,-0.29,-0.47,-1.32,0.71,0.15,-0.62,0.61


In [13]:
portfolios = ['MKT|DAILY_RETURNS',
            '10|DAILY_RETURNS',
#             '10_BROWN_BL|DAILY_RETURNS',
#             '10_BROWN_RO|DAILY_RETURNS',
#             '10_BROWN|DAILY_RETURNS',
#             '10_GREEN_BL|DAILY_RETURNS',
#             '10_GREEN_RO|DAILY_RETURNS',
#             '10_GREEN|DAILY_RETURNS',
            '15|DAILY_RETURNS',
#             '15_BROWN_BL|DAILY_RETURNS',
#             '15_BROWN_RO|DAILY_RETURNS',
#             '15_BROWN|DAILY_RETURNS',
#             '15_GREEN_BL|DAILY_RETURNS',
#             '15_GREEN_RO|DAILY_RETURNS',
#             '15_GREEN|DAILY_RETURNS',
            '20|DAILY_RETURNS',
#             '20_BROWN_BL|DAILY_RETURNS',
#             '20_BROWN_RO|DAILY_RETURNS',
#             '20_BROWN|DAILY_RETURNS',
#             '20_GREEN_BL|DAILY_RETURNS',
#             '20_GREEN_RO|DAILY_RETURNS',
#             '20_GREEN|DAILY_RETURNS',
#             '25|DAILY_RETURNS',
#             '25_BROWN_BL|DAILY_RETURNS',
#             '25_BROWN_RO|DAILY_RETURNS',
#             '25_BROWN|DAILY_RETURNS',
#             '25_GREEN_BL|DAILY_RETURNS',
#             '25_GREEN_RO|DAILY_RETURNS',
#             '25_GREEN|DAILY_RETURNS',
#             '30|DAILY_RETURNS',
#             '30_BROWN_BL|DAILY_RETURNS',
#             '30_BROWN_RO|DAILY_RETURNS',
#             '30_BROWN|DAILY_RETURNS',
#             '30_GREEN_BL|DAILY_RETURNS',
#             '30_GREEN_RO|DAILY_RETURNS',
#             '30_GREEN|DAILY_RETURNS',
            '35|DAILY_RETURNS',
#             '35_BROWN_BL|DAILY_RETURNS',
#             '35_BROWN_RO|DAILY_RETURNS',
#             '35_BROWN|DAILY_RETURNS',
#             '35_GREEN_BL|DAILY_RETURNS',
#             '35_GREEN_RO|DAILY_RETURNS',
#             '35_GREEN|DAILY_RETURNS',
#             '40|DAILY_RETURNS',
#             '40_BROWN_BL|DAILY_RETURNS',
#             '40_BROWN_RO|DAILY_RETURNS',
#             '40_BROWN|DAILY_RETURNS',
#             '40_GREEN_BL|DAILY_RETURNS',
#             '40_GREEN_RO|DAILY_RETURNS',
#             '40_GREEN|DAILY_RETURNS',
#             '45|DAILY_RETURNS',
#             '45_BROWN_BL|DAILY_RETURNS',
#             '45_BROWN_RO|DAILY_RETURNS',
#             '45_BROWN|DAILY_RETURNS',
#             '45_GREEN_BL|DAILY_RETURNS',
#             '45_GREEN_RO|DAILY_RETURNS',
#             '45_GREEN|DAILY_RETURNS',
            '50|DAILY_RETURNS',
#             '50_BROWN_BL|DAILY_RETURNS',
#             '50_BROWN_RO|DAILY_RETURNS',
#             '50_BROWN|DAILY_RETURNS',
#             '50_GREEN_BL|DAILY_RETURNS',
#             '50_GREEN_RO|DAILY_RETURNS',
#             '50_GREEN|DAILY_RETURNS',
#             '55|DAILY_RETURNS',
#             '55_BROWN_BL|DAILY_RETURNS',
#             '55_BROWN_RO|DAILY_RETURNS',
#             '55_BROWN|DAILY_RETURNS',
#             '55_GREEN_BL|DAILY_RETURNS',
#             '55_GREEN_RO|DAILY_RETURNS',
#             '55_GREEN|DAILY_RETURNS',
#             '60|DAILY_RETURNS',
#             '60_BROWN_BL|DAILY_RETURNS',
#             '60_BROWN_RO|DAILY_RETURNS',
#             '60_BROWN|DAILY_RETURNS',
#             '60_GREEN_BL|DAILY_RETURNS',
#             '60_GREEN_RO|DAILY_RETURNS',
#             '60_GREEN|DAILY_RETURNS',
#             'MKT_GREEN_BL|DAILY_RETURNS',
#             'MKT_BROWN_BL|DAILY_RETURNS',
#             'MKT_GMB_BL|DAILY_RETURNS',
#             'MKT_GREEN_RO|DAILY_RETURNS',
#             'MKT_BROWN_RO|DAILY_RETURNS',
#             'MKT_GMB_RO|DAILY_RETURNS',
#             'MKT_BROWN|DAILY_RETURNS',
#             'MKT_GREEN|DAILY_RETURNS',
#             'MKT_GMB|DAILY_RETURNS',
#             'IND_W_GREEN_BL|DAILY_RETURNS',
#             'IND_W_BROWN_BL|DAILY_RETURNS',
#             'IND_W_GMB_BL|DAILY_RETURNS',
#             'IND_W_GREEN_RO|DAILY_RETURNS',
#             'IND_W_BROWN_RO|DAILY_RETURNS',
#             'IND_W_GMB_RO|DAILY_RETURNS',
#             'IND_W_BROWN|DAILY_RETURNS',
#             'IND_W_GREEN|DAILY_RETURNS',
#             'IND_W_GMB|DAILY_RETURNS',
#             'SPX|DAILY_RETURNS',
            'QGREEN|DAILY_RETURNS',
            'SXWESGU|DAILY_RETURNS',
            'RENIXX|DAILY_RETURNS',
            'CELS|DAILY_RETURNS',
            'ECO|DAILY_RETURNS',
            'SPGTCED|DAILY_RETURNS',
            'F3OILG|DAILY_RETURNS',
            'SXEP|DAILY_RETURNS',
            'DJ_OIL_AND_GAS|DAILY_RETURNS']




In [14]:
warnings.filterwarnings("ignore")
for portfolio in portfolios:
    X = merged[['Mkt-RF','COV1_1Jan2021','COV2_1Jan2021']]
    X1= sm.add_constant(X)   
    y = merged[portfolio]-merged['RF']
    model= sm.OLS(y,X1).fit()
    summaryMKT= pd.DataFrame(data=model.summary2().tables[1][["Coef.","P>|t|"]])
    summaryMKT["P>|t|"]=summaryMKT["P>|t|"].apply(significance)
#     summaryMKT["Significance"]=summaryMKT["P>|t|"].apply(significance)
    summaryMKT = summaryMKT.transpose()[["const","Mkt-RF","COV1_1Jan2021","COV2_1Jan2021"]]
    summaryMKT["Adj. R²"]=[model.rsquared_adj,""]
    summaryMKT = pd.concat([summaryMKT], keys=['MKT'], names=['Model'])


    X = merged[['Mkt-RF','SMB','HML','RMW','CMA','COV1_1Jan2021','COV2_1Jan2021']]
    X1= sm.add_constant(X)   
    y = merged[portfolio]-merged['RF']
    model= sm.OLS(y,X1).fit()
    summary5FF= pd.DataFrame(data=model.summary2().tables[1][["Coef.","P>|t|"]])
    summary5FF["P>|t|"]=summary5FF["P>|t|"].apply(significance)
#     summary5FF["Significance"]=summary5FF["P>|t|"].apply(significance)
    summary5FF = summary5FF.transpose()[["const","Mkt-RF","COV1_1Jan2021","COV2_1Jan2021"]]
    summary5FF["Adj. R²"]=[model.rsquared_adj,""]
    summary5FF = pd.concat([summary5FF], keys=['5FF'], names=['Model'])


    X = merged[['Mkt-RF','SMB','HML','RMW','CMA','IND_W_GMB|DAILY_RETURNS','COV1_1Jan2021','COV2_1Jan2021']]
    X1= sm.add_constant(X)   
    y = merged[portfolio]-merged['RF']
    model= sm.OLS(y,X1).fit()
    summary6FF = pd.DataFrame(data=model.summary2().tables[1][["Coef.","P>|t|"]])
    summary6FF["P>|t|"]=summary6FF["P>|t|"].apply(significance)
#     summary6FF["Significance"]=summary6FF["P>|t|"].apply(significance)
    summary6FF = summary6FF.transpose()[["const","Mkt-RF","COV1_1Jan2021","COV2_1Jan2021"]]
    summary6FF["Adj. R²"]=[model.rsquared_adj,""]
    summary6FF = pd.concat([summary6FF], keys=['6FF'], names=['Model'])

    summaryPortfolio = pd.concat([summaryMKT,summary5FF,summary6FF], axis=0)
    summaryPortfolio = pd.concat([summaryPortfolio], keys=[portfolio[:-14]], names=['Portfolio'])
    summaryPortfolio
    
    if portfolio == "MKT|DAILY_RETURNS":
        summary = summaryPortfolio.copy()
    else:
        summary = pd.concat([summary,summaryPortfolio], axis=0)

summary.rename(columns={'const': 'α', 'Mkt-RF': 'β','COV1_1Jan2021': 'δ', 'COV2_1Jan2021': 'γ'}, inplace=True)


In [15]:
pd.set_option('display.max_rows',300)
summary

α         β         δ         γ   Adj. R²
Portfolio      Model                                                        
MKT            MKT   Coef. -0.005654  0.997148  0.001834 -0.018048  0.986379
                     P>|t|                 ***                 ***          
               5FF   Coef. -0.009668  1.008165  0.005635 -0.005771  0.994283
                     P>|t|       ***       ***                              
               6FF   Coef. -0.008972  1.006422  0.005042 -0.006099  0.994502
                     P>|t|       ***       ***                              
10             MKT   Coef.  -0.10233  1.086756  0.222103 -0.067547  0.411837
                     P>|t|         *       ***       ***                    
               5FF   Coef. -0.052365  1.123716  0.123571 -0.049767  0.711733
                     P>|t|                 ***        **                    
               6FF   Coef. -0.050798  1.119797  0.122236 -0.050504  0.711965
                     P>|t|                 ***        **                    
15             MKT   Coef. -0.016311  0.948294  0.018072  0.007355  0.729558
                     P>|t|                 ***                              
               5FF   Coef. -0.002514  0.970068 -0.021265  0.013317  0.786879
                     P>|t|                 ***                              
               6FF   Coef. -0.004576  0.975228 -0.019507  0.014288  0.788289
                     P>|t|                 ***                              
20             MKT   Coef. -0.033121  1.040722  0.032744 -0.109123  0.773257
                     P>|t|                 ***                 ***          
               5FF   Coef. -0.017185  1.054994  -0.01647 -0.093363  0.876015
                     P>|t|                 ***                 ***          
               6FF   Coef. -0.017604  1.056042 -0.016113 -0.093166     0.876
                     P>|t|                 ***                 ***          
35             MKT   Coef.  0.000442  0.875128  0.019755 -0.158672  0.693529
                     P>|t|                 ***                 ***          
               5FF   Coef. -0.007011  0.916991  0.020983 -0.141095  0.723156
                     P>|t|                 ***                 ***          
               6FF   Coef. -0.007337  0.917807  0.021261 -0.140941  0.723032
                     P>|t|                 ***                 ***          
50             MKT   Coef.  0.002356  1.060931 -0.040384 -0.054795  0.706715
                     P>|t|                 ***                              
               5FF   Coef. -0.014853   1.03511  0.002703  -0.04919   0.77116
                     P>|t|                 ***                              
               6FF   Coef. -0.009344  1.021327 -0.001992 -0.051782  0.780298
                     P>|t|                 ***                   *          
QGREEN         MKT   Coef. -0.002746  0.768235  0.010119  0.167157  0.818001
                     P>|t|                 ***                 ***          
               5FF   Coef. -0.000481  0.758052  0.011552  0.154556  0.825411
                     P>|t|                 ***                 ***          
               6FF   Coef. -0.002251  0.762482  0.013062  0.155389  0.826776
                     P>|t|                 ***                 ***          
SXWESGU        MKT   Coef. -0.004197  0.467016 -0.003393  0.134222  0.435485
                     P>|t|                 ***                 ***          
               5FF   Coef.  0.008317  0.462817  -0.02852  0.140482  0.501479
                     P>|t|                 ***                 ***          
               6FF   Coef.  0.008074  0.463425 -0.028312  0.140596  0.501191
                     P>|t|                 ***                 ***          
RENIXX         MKT   Coef.  0.019619  0.523237  0.029821  0.168316  0.182035
                     P>|t|                 ***                  **          
               5FF   Coef.  0.0

In [16]:
summary.to_excel(dataPath+"/processed/regressionSummary.xlsx")